In [1]:
import itertools
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.neighbors import NearestNeighbors

from node2vec import Node2Vec

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler, Sampler
from torch.utils.data import Dataset

from torch_geometric.utils.convert import from_networkx

In [37]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    # lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

haversine = np.vectorize(haversine)

In [38]:
nodes_df = pd.read_csv('data/road_intersection_nodes.csv')
nodes_df

,lng,lat,id
0,-74.017931,40.706175,0
1,-74.017869,40.706349,1
2,-74.017789,40.706519,2
3,-74.017690,40.706683,3
4,-74.017574,40.706840,4
...,...,...,...
236253,-73.901365,40.663609,2414372
236254,-73.951336,40.742705,2414375
236255,-73.951368,40.742617,2414376
236256,-73.951404,40.742530,2414377


In [39]:
nodes_df[['lng', 'lat']] = nodes_df[['lng', 'lat']].apply(np.vectorize(radians))
nodes_df.head()

,lng,lat,id
0,-1.291857,0.710457,0
1,-1.291856,0.710460,1
2,-1.291854,0.710463,2
3,-1.291852,0.710466,3
4,-1.291850,0.710468,4


In [40]:
lat_center, lng_center = nodes_df.lat.mean(), nodes_df.lng.mean()
lat_center, lng_center

(0.7106174200098879, -1.290224335366838)

In [87]:
RADIUS = 8

obs_nodes_df = nodes_df[haversine(lng_center, lat_center, nodes_df.lng, nodes_df.lat) <= RADIUS]
obs_nodes_set = set(obs_nodes_df.id.unique())

In [88]:
pickups_df = pd.read_csv('data/TLC_daily.csv')
pickups_df = pickups_df[pickups_df['id'].isin(obs_nodes_set)]
pickups_df

,day,id,pickups
0,1.0,0,19.0
1,1.0,1,18.0
2,1.0,2,18.0
3,1.0,3,17.0
4,1.0,4,13.0
...,...,...,...
35911210,152.0,2414253,0.0
35911211,152.0,2414254,0.0
35911212,152.0,2414267,0.0
35911213,152.0,2414268,0.0


In [89]:
edges_df = pd.read_csv('data/road_intersection_edges.csv')
edges_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282983 entries, 0 to 282982
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   olng    282983 non-null  float64
 1   olat    282983 non-null  float64
 2   dlng    282983 non-null  float64
 3   dlat    282983 non-null  float64
 4   oid     282983 non-null  int64  
 5   did     282983 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 13.0 MB


In [90]:
obs_edges_df = edges_df[edges_df.oid.isin(obs_nodes_set) & edges_df.did.isin(obs_nodes_set)]
obs_edges_df['dist'] = obs_edges_df.apply(lambda x: haversine(x['olng'], x['olat'], x['dlng'], x['dlat']), axis=1).astype(float)

/tmp/ipykernel_2846/2957988538.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_edges_df['dist'] = obs_edges_df.apply(lambda x: haversine(x['olng'], x['olat'], x['dlng'], x['dlat']), axis=1).astype(float)


In [91]:
def make_graph_from_df(nodes_df, edges_df, name='TLC', directed=False):
    G = nx.Graph(directed=directed)
    G.graph['Name'] = name

    G.add_nodes_from(nodes_df.set_index('id').to_dict('index').items())
    G.add_nodes_from((n, {'id': n}) for n in G.nodes())

    G.add_edges_from(nx.from_pandas_edgelist(edges_df, 'oid', 'did', ['dist']).edges(data=True))

    return G

G = make_graph_from_df(obs_nodes_df, obs_edges_df)
G.number_of_nodes(), G.number_of_edges()

(60789, 75647)

In [92]:
embedder = Node2Vec(G, dimensions=128, walk_length=30, workers=4)

Generating walks (CPU: 4): 100%|██████████| 2/2 [00:13<00:00,  6.68s/it]


In [93]:
model = embedder.fit(window=5, min_count=1)

In [94]:
for idx, e in enumerate(G.edges()):
    if idx == 5:
        break
    print(e)

(0, 1)
(0, 24)
(0, 350192)
(1, 2)
(2, 3)


In [95]:
model.wv.most_similar('0')

[('1', 0.990696907043457),
 ('350192', 0.9867452383041382),
 ('2', 0.974662184715271),
 ('24', 0.9740908741950989),
 ('200921', 0.9610595703125),
 ('3', 0.9553714394569397),
 ('350191', 0.9537723064422607),
 ('211643', 0.9374097585678101),
 ('332388', 0.9370771646499634),
 ('200923', 0.9347763061523438)]

In [96]:
for idx, node in enumerate(G.nodes()):
    G.add_node(int(node), x=model.wv[str(node)].copy())

In [148]:
class DayObservationsDataset(Dataset):
    def __init__(self, df: pd.DataFrame) -> None:
        super().__init__()
        self.data = df['id'].to_numpy()
        self.targets = df['pickups'].to_numpy()
        self.observed_nodes = set(np.unique(self.data))

        self.node_to_target = df.set_index('id')['pickups'].to_dict()
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # returns node, and its observation
        return self.data[idx], self.targets[idx]

    def get_observation_by_node(self, node):
        return self.node_to_target[node]

    def get_observed_nodes(self):
        return self.observed_nodes


def get_dataset_by_day(pickups_df, day):
    df = pickups_df[pickups_df['day'].astype('int') == day].copy()
    df.drop('day', axis=1, inplace=True)
    return DayObservationsDataset(df)


ds = get_dataset_by_day(pickups_df, 25)
len(ds)

60789

In [149]:
np.random.seed(228)

indices = list(range(len(ds)))

train_indices, test_indices = train_test_split(indices, test_size=0.3)
train_indices, val_indices = train_test_split(train_indices, test_size=0.3)
print(len(train_indices), len(val_indices), len(test_indices))

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)


train_loader = DataLoader(ds, batch_size=128, sampler=train_sampler)
val_loader = DataLoader(ds, batch_size=len(val_sampler.indices), sampler=val_sampler)
test_loader = DataLoader(ds, batch_size=len(test_sampler.indices), sampler=test_sampler)

29786 12766 18237


In [150]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [151]:
pyg = from_networkx(G)
pyg.to(device)

Data(x=[60789, 128], edge_index=[2, 151294], lng=[60789], lat=[60789], id=[60789], dist=[151294])

In [154]:
def weight_fn(dists, lamb):
    return torch.exp(-lamb * dists)


class Predictor(nn.Module):
    def __init__(self, pyg_graph, nodes_df, observs) -> None:
        super().__init__()

        self.g = pyg_graph
        self.nodes_df = nodes_df
        self.obs_nodes = observs[0]
        self.obs_targets = observs[1]

        self.NEIGHBORS_NUM = 15
        
        self.node_to_idx = np.vectorize(dict(zip(self.nodes_df.id.values, itertools.count())).get)
        self.node_to_gidx = np.vectorize(dict(zip(self.g.id.detach().cpu().numpy(), range(len(self.g.id)))).get)

        self.neighbors = NearestNeighbors(n_neighbors=self.NEIGHBORS_NUM, metric='haversine')
        self.obs_nodes_locs = self.nodes_df.iloc[self.node_to_idx(self.obs_nodes)]
        self.neighbors.fit(self.obs_nodes_locs[['lat', 'lng']].values)

        self.k = nn.Parameter(torch.rand(1))
        # self.k = torch.tensor([1.0])
        self.lambda_1 = nn.Parameter(torch.rand(1))
        self.lambda_2 = nn.Parameter(torch.rand(1))

    def forward(self, X):
        # getting nearest observed nodes
        dists, indices = self.neighbors.kneighbors(self.nodes_df.iloc[self.node_to_idx(X.detach().cpu())][['lat', 'lng']].values)
        dists = dists * 6371 * 1000

        # so for now dist in meters
        if self.training:
            dists, indices = dists[:, 1:], indices[:, 1:]
        # get_observations = np.vectorize(self.obs_ds.get_observation_by_node)
        # observations = get_observations(self.obs_nodes_locs['id'].values[indices])
        observations = self.obs_targets[indices]

        dists, observations = torch.as_tensor(dists).to(device), torch.as_tensor(observations).to(device)

        neighbors_indices = self.node_to_gidx(self.obs_nodes[indices])
        neighbors_embeds = self.g.x[neighbors_indices.reshape(-1)].reshape(*neighbors_indices.shape, -1)

        X_embeds = self.g.x[self.node_to_gidx(X.detach().cpu())]
        similarities = nn.functional.cosine_similarity(X_embeds[:, None], neighbors_embeds, dim=2)
        # similarities = torch.sum(X_embeds[:, None] * neighbors_embeds, dim=-1)
        # print(similarities)

        dist_weights = weight_fn(dists, self.lambda_1)
        simi_weights = weight_fn(similarities, self.lambda_2)
        # sum normalizization
        dist_weights = nn.functional.normalize(dist_weights, p=1)
        simi_weights = nn.functional.normalize(simi_weights, p=1)

        f = self.k * dist_weights + (1 - self.k) * simi_weights

        result = torch.sum(f.mul(observations), dim=-1)

        return result

predictor = Predictor(pyg, obs_nodes_df, ds[train_indices]).to(device)
# kek = next(iter(train_loader))
# predictor(kek[0])[:20], kek[1][:20] 

In [157]:
loss_fn = nn.HuberLoss(delta=20).to(device)
# loss_fn = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(predictor.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.5)


def calc_score(pred, actual):
    return r2_score(actual, pred)

def test(model, loader):
    model.eval()

    scores = []
    total_loss = 0

    with torch.no_grad():
        for (X, y) in loader:
            X_gpu = X.to(device)
            y_gpu = y.to(device)
            out = model(X_gpu)
            scores.append(calc_score(out.detach().cpu(), y))
            loss = loss_fn(out, y_gpu)
            total_loss += loss.item()
    
    return total_loss / len(loader), np.mean(scores)


def train(model, train_loader, val_loader, loss_fn, optimizer, scheduler=None, num_epochs=30):
    losses = []
    test_scores = []
    

    for epoch in range(num_epochs + 1):
        model.train()
        total_loss = 0

        for i_step, (X, y) in enumerate(train_loader):
            optimizer.zero_grad()
            X_gpu = X.to(device)
            y_gpu = y.to(device)
            out = model(X_gpu)
            loss = loss_fn(out, y_gpu)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        total_loss /= len(train_loader)
        losses.append(total_loss)

        if scheduler is not None:
            scheduler.step()

        if epoch % 5 == 0:
            val_loss, score = test(model, val_loader)
            test_scores.append(score)
            print(f'Epoch {epoch}, Loss: {losses[-1]:.4f}, Val loss: {val_loss:.4f}, Val R2: {test_scores[-1]:.4f}')

train(predictor, train_loader, val_loader, loss_fn, optimizer, scheduler)
for name, param in predictor.named_parameters():
    print(name, param)

print(test(predictor, test_loader))

Epoch 0, Loss: 59.7489, Val loss: 62.3918, Val R2: 0.8208
Epoch 5, Loss: 53.6995, Val loss: 48.4328, Val R2: 0.8724
Epoch 10, Loss: 44.3718, Val loss: 43.5485, Val R2: 0.8878
Epoch 15, Loss: 44.4821, Val loss: 43.1037, Val R2: 0.8875
Epoch 20, Loss: 44.1363, Val loss: 43.2139, Val R2: 0.8876
Epoch 25, Loss: 44.0865, Val loss: 43.1916, Val R2: 0.8875
Epoch 30, Loss: 44.1232, Val loss: 43.1901, Val R2: 0.8875
k Parameter containing:
tensor([0.8893], device='cuda:0', requires_grad=True)
lambda_1 Parameter containing:
tensor([0.1082], device='cuda:0', requires_grad=True)
lambda_2 Parameter containing:
tensor([-16.8869], device='cuda:0', requires_grad=True)
(46.44965473775957, 0.8798796661120525)
